In [48]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Bidirectional, LSTM, SimpleRNN
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import GRU

In [49]:
# Load the dataset
df = pd.read_csv('..//dataset//final_dataset_compressed.csv')  # Replace with your dataset path

# Separating features and target
X = df[['Number of Lanes', 'Status of driver', 'Nature of environment', 'Velocity of vehicle', 'Distance between vehicles', 'Breaking capability']]
y = df['Collussion']

# Normalize the features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)


from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)
# X_scaled = X

# Reshape input to be 3D [samples, timesteps, features] for LSTM
X_scaled = np.reshape(X_scaled, (X_scaled.shape[0], 1, X_scaled.shape[1]))

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [50]:
print(X_train.shape, y_train.shape,X_scaled.shape)


(320000, 1, 6) (320000,) (400000, 1, 6)


# Model

In [51]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Conv1D, Flatten
from tensorflow.keras.layers import BatchNormalization
def build_deep_rnn_bi_lstm():
      model = Sequential()

      # Deep RNN layers
      model.add(SimpleRNN(units=50, return_sequences=True,
                           input_shape=(X_train.shape[1], X_train.shape[2]),
                           kernel_regularizer=l2(0.01)))
      model.add(BatchNormalization())
      model.add(Dropout(0.3))
      model.add(SimpleRNN(units=50, return_sequences=True, kernel_regularizer=l2(0.01)))
      model.add(BatchNormalization())
      model.add(Dropout(0.3))

      # Bidirectional LSTM layers
      model.add(Bidirectional(LSTM(units=50, return_sequences=True, kernel_regularizer=l2(0.01))))
      model.add(BatchNormalization())
      model.add(Dropout(0.3))
      model.add(Bidirectional(LSTM(units=50, return_sequences=True, kernel_regularizer=l2(0.01))))
      model.add(BatchNormalization())
      model.add(Dropout(0.3))
      model.add(Bidirectional(LSTM(units=50, kernel_regularizer=l2(0.01))))
      model.add(BatchNormalization())
      model.add(Dropout(0.3))

      # Output layer
      model.add(Dense(units=1, activation='sigmoid'))

      # Compile the model with Adam optimizer (adjust learning rate)
      model.compile(optimizer=Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['mse', 'mae'])

      return model



In [52]:
# Instantiate and train the model
model = build_deep_rnn_bi_lstm()
history = model.fit(
    X_train, 
    y_train, 
    epochs=5, 
    batch_size=64, 
    validation_split=0.2, 
    verbose=1, 
)

d:\study\codes\engineering_codes\research\conda_env_for_project\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 87s 15ms/step - loss: 1.8455 - mae: 0.0925 - mse: 0.0515 - val_loss: 0.0546 - val_mae: 0.0278 - val_mse: 0.0140
Epoch 2/5
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 40s 10ms/step - loss: 0.0759 - mae: 0.0389 - mse: 0.0197 - val_loss: 0.0688 - val_mae: 0.0442 - val_mse: 0.0162
Epoch 3/5
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 37s 9ms/step - loss: 0.0728 - mae: 0.0377 - mse: 0.0190 - val_loss: 0.0558 - val_mae: 0.0285 - val_mse: 0.0153
Epoch 4/5
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 38s 10ms/step - loss: 0.0715 - mae: 0.0370 - mse: 0.0188 - val_loss: 0.0824 - val_mae: 0.0313 - val_mse: 0.0214
Epoch 5/5
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 36s 9ms/step - loss: 0.0692 - mae: 0.0357 - mse: 0.0180 - val_loss: 0.0534 - val_mae: 0.0232 - val_mse: 0.0138


In [53]:
import numpy as np

# Set numpy to display full float numbers
np.set_printoptions(suppress=True, precision=10)

In [54]:
# Evaluate the model
# results = model.evaluate(X_test, y_test, verbose=1)
# print(results)


In [55]:

from sklearn.metrics import mean_squared_error
import numpy as np

# Make predictions
y_pred = model.predict(X_test)

# Calculate MSE
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error (MSE): {mse:.16f}')

# Calculate RMSE
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE): {rmse:.16f}')

2500/2500 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step
Mean Squared Error (MSE): 0.0138694429770112
Root Mean Squared Error (RMSE): 0.1177685992827086


In [56]:
# #visualizing the loss and accuracy
# import matplotlib.pyplot as plt
# plt.figure(figsize=(10,5))
# plt.subplot(1,2,1)
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('Model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train','validation'],loc='upper left')

# plt.subplot(1,2,2)
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train','validation'],loc='upper left')
# plt.show()
